<a href="https://colab.research.google.com/github/inspire-lab/CyberAI-labs/blob/main/category-SecureAI/Hate-speech-offensive-language-classification/hatespeech_Vs_offensive_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4
## Classifying between hate speech and offensive language.

In this lab we will learn to classify hate speech versus offensive language. Specifically, we will use a social media corpus consisting of tweets which are labeled as hate_speech, offensive_language or neither.

We will try an approach similar to what we followed in Lab 1 to see how the results are. Then, we will understand what word embeddings are, and use the popular glove pre-trained word embeddings in our model. Later, we will train our own word embeddings to use in our model and compare the results between the two.

The task itself is not as straightforward as it might appear at first glance. Just think about it. There is a big difference between between `You are f****ing great` and `You are f****ing @*%&!*@#%` (replace `@*%&!*@#%` with your favorite worst insult). We can all agree on the first sentence not being hate speech. But some people might consider offensive language. The problem is very subjective.

This is reflected in the data labeling. The creators of the data set gave the tweets to multiple annotators (Annotators are people who assign labels to data, roughly speaking) and the label would be what the most annotators asigned.


In [ ]:
# this lab needs tensorflow 2.8.0
!pip install tensorflow==2.8.0

## Question 1:

Let us first understand our task. Given the tweets below,categorize eachof these tweets into one of the category
#### -hate_speech
#### -offensive_language
#### -neither

1. "@Blackman38Tide: @WhaleLookyHere @HowdyDowdy11 queer" gaywad

2. "I'll slit your throat-neck"

3. "Roberto you are one nasty hoe"

4. &#8220;@CommonBlackGirI: here&#8217;s to all the kids who have never found their name on anything in a souvenir store&#8221; thanks mom for the ghetto name.

5. &#8220;@Damn_ImTall: Anybody that gets traded straight up for Brandon knight is trash @BrandonJennings&#8221;@BushALMIGHTY

6. &#8220;@DickFurari: HELL YA THEY SHOULD MAKE MORE MALT LIQUOR ADS LIKE THIS. http://t.co/H0JGORtnF8&#8221;- Dis be heaven fo a colored man

Your Answers:

1.

2.

3.

4.

5.

6.

You would have now understood how ambiguous it is to decide the label of each tweet is. To see what the labels in the dataset are, scroll to the bottom of this page.

The first step is to load the data into a data frame.

In [ ]:
import pandas as pd

col_names = [
    'sno', 'count', 'hate_speech', 'offensive_language', 'neither', 'class',
    'tweet'
]
df = pd.read_csv('data/hate_offensive.csv', skiprows=[0], names=col_names)


Now we define a function tweet_cleaner to preprocess the tweets.

## Question 2:
The function tweet_cleaner misses the statement to convert the tweets to lower case. Insert the required code in the specified location. Look at what else the function does.

In [ ]:
# solution
from nltk.tokenize import WordPunctTokenizer

tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

from bs4 import BeautifulSoup
import re


def tweet_cleaner(text):
  soup = BeautifulSoup(text, 'lxml')
  souped = soup.get_text()
  stripped = re.sub(combined_pat, '', souped)
  try:
    clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
  except:
    clean = stripped
  letters_only = re.sub("[^a-zA-Z]", " ", clean)
  #######################
  lower_case = # complete the code
  #######################
  lower_case = letters_only.lower()
  words = tok.tokenize(lower_case)
  return (" ".join(words)).strip()

Call the function tweet_cleaner to clean the tweets.

In [ ]:
df['tweet'] = df['tweet'].apply(tweet_cleaner)

The dataset we use has around 25000 tweets. It will take a long time to train during the lab session, given our system configuration. So we will use the top 2000 tweets for our training. Print the top five tweets to see how the preprocessed tweets look like.

In [ ]:
print(df.tweet.head())


## Question 3:
Create a 80:20 train test split from the dataframe df.

In [ ]:
# solution
from sklearn.model_selection import train_test_split
#######################
train, test = # complete the code
#######################
train, test = train_test_split(df, test_size=0.2, stratify=df['class'])
targets = train['class']

Let us prepare our tweets for the bag of words model and then set up a majority baseline model

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
# CountVectorizer-Convert a collection of text documents to a matrix of token counts
count_vectorizer = CountVectorizer(analyzer='word', max_features=500)
train_bog = count_vectorizer.fit_transform(train['tweet'].values)
test_bog = count_vectorizer.transform(test['tweet'])

from sklearn.dummy import DummyClassifier

base = DummyClassifier(strategy='most_frequent')
base.fit(train_bog, targets)
base_predictions = base.predict(test_bog)

## Question 4:
Print the classification report for the majority baseline.

In [ ]:
# solution


## Question 5:
Implement a naive bayes bag of words model using the given train data.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
#######################
# complete the code

# holds the predictions of the naive bayes bag of words model
bog_bayes_predictions =
#######################

print(
    classification_report(test['class'],
                          bog_bayes_predictions,
                          labels=[0, 1, 2]))


## Question 6:
Implement a naive bayes TF-IDF model using the train data.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#######################
# complete the code

# holds the predictions of the tf-idf model
tfidf_bayes_predictions =
#######################
print(
    classification_report(test['class'],
                          tfidf_bayes_predictions,
                          labels=[0, 1, 2]))


## Question 7:
Implement a naive bayes bigram model using the training data.

In [ ]:
# solution


Now that we have tried the traditional learning methods, let us try some neural learning architectures using word embeddings.
We will use keras to implement neural network architectures. Read more on keras <a href = "https://keras.io/"> here </a>.

In [ ]:
!pip install protobuf==3.20.3

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Dropout, Activation
from keras.layers.embeddings import Embedding

Before we can use the embeddings we need to download the pretrainted word vectors. https://nlp.stanford.edu/projects/glove/

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

Word embeddings are a representation of words as numbers, as neural networks can understand only numbers. Learn more about word embeddings <a href="https://machinelearningmastery.com/what-are-word-embeddings/">here</a>.

The code below loads the glove embeddings into a dictionary.

In [ ]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

In the code block below, we tokenize the tweets in our dataset and create indices for each word in the vocabulary. We then make the length of the tweets equal.To do so, we will the length of the longest tweet nd append all other tweets with padding.

In [ ]:
# download additional ressrouces
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tweet = df.tweet
word_count = lambda tweet: len(word_tokenize(tweet))

max_len_tweet = len(max(tweet, key=word_count))
t = Tokenizer()
t.fit_on_texts(tweet)
vocabulary_size = len(t.word_index) + 1
encoded_tweet = t.texts_to_sequences(tweet)
padded_tweets = pad_sequences(encoded_tweet,
                              maxlen=max_len_tweet,
                              padding='post')

We will create an embedding matrix which will load the glove embeddings for the words in our vocabuary from the dictionary embeddings_index.

In [ ]:
embedding_size = 100
embedding_matrix = np.zeros((vocabulary_size, embedding_size))
for word, index in t.word_index.items():
  if index > vocabulary_size - 1:
    break
  else:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[index] = embedding_vector


Clearly, we have loaded the embeddings for the entire dataset.

## Question 8:
Create a 80:20 train test split over the dataset that are represented as the indices in the embedding_index dictionary. Ensure that the train test instances in the previous split is the same as in the current split.

In [ ]:
# solution


Let us now create a simple model using the glove embeddings with one LSTM layer. Do not worry about the parameters of the architecture for now.

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense

model_glove = Sequential()
model_glove.add(
    Embedding(vocabulary_size,
              embedding_size,
              weights=[embedding_matrix],
              input_length=max_len_tweet,
              trainable=False))
model_glove.add(LSTM(100, dropout=0.5, recurrent_dropout=0.5))
model_glove.add(Dropout(0.3))
model_glove.add(Dense(3, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])
model_glove.summary()


Let us train the model using a 20% validation split from the training set. The objective is to increase the accuracy and decrease the loss. Play with the different hyper parameters (epochs, batch_size) to see if there is a change in the accuracy and loss values.

In [ ]:
model_glove.fit(train_x,
                train_y,
                validation_split=0.2,
                epochs=2,
                batch_size=32,
                verbose=1)


In [ ]:
predictions = model_glove.predict(test_x)
predictions = to_categorical(np.argmax(predictions, axis=1), num_classes=3)
print(classification_report(test_y, predictions))


To examine if our model performs better with a different configuration, we can increase the number of epochs. However, this could also increase the run time. Hence we will load the model which used 20 epochs instead of 2 epochs over the same architecture.

In [ ]:
#loading the saved gloved model with 20 epochs
loaded_glove_model = load_model('glove_models/glove_model.hdf5')

In [ ]:
predictions = loaded_glove_model.predict(test_x)
predictions = to_categorical(np.argmax(predictions, axis=1), num_classes=3)
print(classification_report(test_y, predictions))

The classification report shows that even after running for 20 epochs, there is no improvement in performance, and all tweets are labeled as offensive_langugae which is our majority label.
This indicates that either the architecture of the neural network is too simple, or the way we represent the input (word embeddings) needs modification, or we need further hyper-parameter tuning.

Let us now train our own word embeddings using the Word2Vec function available in the package gensim. Read more abour Word2Vec <a href="https://skymind.ai/wiki/word2vec">here</a>. Read more on Word2Vec on gensim <a href="https://rare-technologies.com/word2vec-tutorial/">here</a>.

In [ ]:
from nltk import word_tokenize
from gensim.models import Word2Vec

df['tokenized_tweet'] = df["tweet"].apply(word_tokenize)

gensim_model = Word2Vec(df['tokenized_tweet'],
                        vector_size=100,
                        min_count=1,
                        window=5,
                        epochs=100)
gensim_trained_weights = gensim_model.wv.vectors
gensim_vocabulary_size, gensim_emdedding_size = gensim_trained_weights.shape


def word2idx(word):
  return gensim_model.wv.vocab[word].index


def idx2word(idx):
  return gensim_model.wv.index2word[idx]

Let us now create a train test split over the tokenized tweet and then load the embeddings that we trained using Word2Ved in gensim.

In [ ]:
# Ensure word2idx uses key_to_index for compatibility with Gensim 4.x
def word2idx(word):
    # Use .get(word, 0) to return 0 for out-of-vocabulary words
    return gensim_model.wv.key_to_index.get(word, 0)

# Initialize arrays for train and test data
word2vec_train_x = np.zeros([len(train_x), max_len_tweet], dtype=np.int32)
word2vec_train_y = np.zeros([len(train_y)], dtype=np.int32)
word2vec_test_x = np.zeros([len(test_x), max_len_tweet], dtype=np.int32)
word2vec_test_y = np.zeros([len(test_y)], dtype=np.int32)

# Encode train set
for i, tweet in enumerate(train_x):
    for t, word in enumerate(tweet):
        if t < max_len_tweet:
            word2vec_train_x[i, t] = word2idx(word)  # This should now work without referencing .vocab

# Encode test set
for i, tweet in enumerate(test_x):
    for t, word in enumerate(tweet):
        if t < max_len_tweet:
            word2vec_test_x[i, t] = word2idx(word)


Let us now plot our word embeddings over a graph to understand how the words are related.

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Access the vocabulary and vectors
vocab = list(gensim_model.wv.index_to_key)  # Use .index_to_key to get the list of words
X = gensim_model.wv[vocab]  # Get the vectors for each word in the vocabulary

# Perform t-SNE dimensionality reduction
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)

# Create a DataFrame with the 2D t-SNE coordinates
d = pd.DataFrame(X_tsne, index=vocab, columns=['x', 'y'])


For simplicity, we will project the words from the top 1000 instances of the data set.

In [ ]:
d = d.head(1000)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.scatter(d['x'], d['y'])
plt.figure(figsize=(500, 500))
for word, pos in d.iterrows():
  ax.annotate(word, pos)

plt.show()

## Question 9:
Build a model using the same architecture and use the embeddings that we generated using Word2Vec from gensim.

## Question 10:
Train the architecture using the same split with our own embeddings and print the classification report.

As we can see, there is still no improvement in performance.

## Question 11:
Load the model trained over the same architecture using the embeddings trained over 20 epochs. The model is in the location `word2vec_models/w2v_model.hdf5`. Print the classification report for this model.

The results imply that the architecture built over the embeddings that we trained on the tweets perform better than using the existing pretrained glove embeddings. Hence it is necessary to build task specific models.

## Question 12:
Comment of why the embeddings we trained using Word2Vec are better than the pretrained glove embeddings over this particular task of classify hate_speech, offensive_language or neither.

Your Answer:

